In [1]:
# Imports
import cv2 
# USE PYTHON 3.11

In [2]:
# Set up Haar-Casacades

# Load classifier
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
def detect_bounding_boxes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(20, 20))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
    
    return faces

In [ ]:
# Start up web-cam
capture = cv2.VideoCapture(1) # 0 for laptop camera, otherwise provide path to video

# Continuously read frames and apply haar cascades
while True:
    result, frame = capture.read() # Read a frame from the video

    if not result:
        print("ERROR! Breaking")
        break 

    faces = detect_bounding_boxes(frame)

    cv2.imshow("Detection", frame) # Display processed frame in a window

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break 

capture.release()
cv2.destroyAllWindows()

2025-10-14 18:25:11.997 Python[45019:3192477] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


: 